# Stock Price Prediction - Model Performance Analysis & Comparison

**Objective**: Analyze and compare different machine learning models for stock price prediction

**Models Evaluated**:
- Linear Regression
- Polynomial Regression (degree 2)
- Ridge Regression
- Lasso Regression
- Decision Tree Regressor
- Random Forest Regressor

**Evaluation Metrics**:
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- Mean Absolute Error (MAE)
- R² Score
- Mean Absolute Percentage Error (MAPE)

## 1. Import Libraries and Load Data

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [2]:
# Load the dataset with technical indicators
df = pd.read_csv('AAPL_60d_5min_with_indicators.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()[:10]}...")
print(f"\nFirst few rows:")
df.head()

Dataset Shape: (4578, 92)

Columns: ['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi']...

First few rows:


,Datetime,Close,High,Low,Open,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2025-06-27 13:30:00+00:00,202.839905,203.220001,201.259995,201.895004,3499442,2.142176e+06,3499442,0.612148,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,202.839905,0.000000,0.000000,0.000000
1,2025-06-27 13:35:00+00:00,202.285004,202.889999,202.139999,202.813400,1001258,1.528082e+06,2498184,0.339521,-555599.188660,...,-0.021827,-0.004365,-0.017462,-6.012728,-1.202546,-4.810182,202.611606,-0.273566,-0.273941,-0.273566
2,2025-06-27 13:40:00+00:00,202.190002,202.529999,201.929993,202.300003,710984,1.433301e+06,1787200,0.275017,-485877.068551,...,-0.042426,-0.011978,-0.030449,-12.055092,-3.373055,-8.682037,202.433093,-0.046964,-0.046975,-0.320402
3,2025-06-27 13:45:00+00:00,202.505005,202.580002,202.090103,202.225006,626628,1.868073e+06,2413828,0.319968,-388267.580208,...,-0.045688,-0.018720,-0.026968,-17.619449,-6.222334,-11.397115,202.463041,0.155795,0.155674,-0.165106
4,2025-06-27 13:50:00+00:00,202.839996,203.130005,202.445404,202.505005,844804,1.997130e+06,3258632,0.298832,-292371.908576,...,-0.034538,-0.021883,-0.012654,-21.738214,-9.325510,-12.412704,202.619891,0.165424,0.165287,0.000045


## 2. Data Preparation

In [3]:
# Create target variable (price 10 minutes ahead = 2 bars ahead)
df['Target'] = df['Close'].shift(-2)

# Drop rows with NaN
df_clean = df.dropna()

print(f"Clean Dataset Shape: {df_clean.shape}")
print(f"Rows removed: {len(df) - len(df_clean)}")

Clean Dataset Shape: (4576, 93)
Rows removed: 2


In [4]:
# Select features (using the same 16 features as the current model)
features = [
    'Volume', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
    'trend_macd_diff', 'trend_adx', 'momentum_rsi', 'momentum_stoch_rsi_k',
    'momentum_stoch_rsi_d', 'momentum_roc', 'volatility_atr', 'volatility_bbw',
    'volume_obv', 'volume_vwap', 'volume_mfi'
]

# Prepare X and y
X = df_clean[features].copy()
y = df_clean['Target'].copy()

# Transform Volume (log transformation)
X['Volume'] = np.log1p(X['Volume'])

print(f"Features: {len(features)}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

Features: 16
X shape: (4576, 16)
y shape: (4576,)


In [5]:
# Split data (80-20 split for time series)
split_idx = int(len(X) * 0.8)

X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]
y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Split ratio: {X_train.shape[0]/len(X)*100:.1f}% train, {X_test.shape[0]/len(X)*100:.1f}% test")

Training set: 3660 samples
Test set: 916 samples
Split ratio: 80.0% train, 20.0% test


In [6]:
# Scale the features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

print("✓ Data scaled successfully!")

✓ Data scaled successfully!


## 3. Model Training & Evaluation

In [7]:
# Function to calculate all metrics
def calculate_metrics(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R² Score': r2,
        'MAPE (%)': mape
    }

# Dictionary to store models and results
models = {}
results = []
predictions = {}

print("Starting model training...\n")

Starting model training...



### 3.1 Linear Regression

In [8]:
print("Training Linear Regression...")

# Train model
lr = LinearRegression()
lr.fit(X_train_scaled, y_train_scaled)

# Predict
y_pred_scaled = lr.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# Store
models['Linear Regression'] = lr
predictions['Linear Regression'] = y_pred
results.append(calculate_metrics(y_test, y_pred, 'Linear Regression'))

print(f"✓ Linear Regression trained")
print(f"  R² Score: {results[-1]['R² Score']:.4f}")
print(f"  RMSE: ${results[-1]['RMSE']:.4f}")

Training Linear Regression...
✓ Linear Regression trained
  R² Score: 0.9851
  RMSE: $0.5900


### 3.2 Polynomial Regression (Degree 2)

In [9]:
print("\nTraining Polynomial Regression (degree=2)...")

# Create polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Train model
poly_lr = LinearRegression()
poly_lr.fit(X_train_poly, y_train_scaled)

# Predict
y_pred_scaled = poly_lr.predict(X_test_poly)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# Store
models['Polynomial Regression'] = (poly, poly_lr)
predictions['Polynomial Regression'] = y_pred
results.append(calculate_metrics(y_test, y_pred, 'Polynomial Regression'))

print(f"✓ Polynomial Regression trained")
print(f"  R² Score: {results[-1]['R² Score']:.4f}")
print(f"  RMSE: ${results[-1]['RMSE']:.4f}")


Training Polynomial Regression (degree=2)...
✓ Polynomial Regression trained
  R² Score: 0.9815
  RMSE: $0.6576


### 3.3 Ridge Regression

In [10]:
print("\nTraining Ridge Regression...")

# Train model with different alpha values and select best
best_alpha = None
best_score = -np.inf

for alpha in [0.01, 0.1, 1.0, 10.0, 100.0]:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train_scaled)
    score = ridge.score(X_test_scaled, y_test_scaled)
    if score > best_score:
        best_score = score
        best_alpha = alpha
        best_ridge = ridge

print(f"  Best alpha: {best_alpha}")

# Predict
y_pred_scaled = best_ridge.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# Store
models['Ridge Regression'] = best_ridge
predictions['Ridge Regression'] = y_pred
results.append(calculate_metrics(y_test, y_pred, 'Ridge Regression'))

print(f"✓ Ridge Regression trained")
print(f"  R² Score: {results[-1]['R² Score']:.4f}")
print(f"  RMSE: ${results[-1]['RMSE']:.4f}")


Training Ridge Regression...
  Best alpha: 0.01
✓ Ridge Regression trained
  R² Score: 0.9851
  RMSE: $0.5906


### 3.4 Lasso Regression

In [11]:
print("\nTraining Lasso Regression...")

# Train model with different alpha values and select best
best_alpha = None
best_score = -np.inf

for alpha in [0.0001, 0.001, 0.01, 0.1, 1.0]:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train_scaled, y_train_scaled)
    score = lasso.score(X_test_scaled, y_test_scaled)
    if score > best_score:
        best_score = score
        best_alpha = alpha
        best_lasso = lasso

print(f"  Best alpha: {best_alpha}")

# Predict
y_pred_scaled = best_lasso.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# Store
models['Lasso Regression'] = best_lasso
predictions['Lasso Regression'] = y_pred
results.append(calculate_metrics(y_test, y_pred, 'Lasso Regression'))

print(f"✓ Lasso Regression trained")
print(f"  R² Score: {results[-1]['R² Score']:.4f}")
print(f"  RMSE: ${results[-1]['RMSE']:.4f}")


Training Lasso Regression...
  Best alpha: 0.0001
✓ Lasso Regression trained
  R² Score: 0.9845
  RMSE: $0.6022
  Best alpha: 0.0001
✓ Lasso Regression trained
  R² Score: 0.9845
  RMSE: $0.6022


### 3.5 Decision Tree Regressor

In [12]:
print("\nTraining Decision Tree Regressor...")

# Train with optimal parameters to prevent overfitting
dt = DecisionTreeRegressor(
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)
dt.fit(X_train_scaled, y_train_scaled)

# Predict
y_pred_scaled = dt.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# Store
models['Decision Tree'] = dt
predictions['Decision Tree'] = y_pred
results.append(calculate_metrics(y_test, y_pred, 'Decision Tree'))

print(f"✓ Decision Tree trained")
print(f"  R² Score: {results[-1]['R² Score']:.4f}")
print(f"  RMSE: ${results[-1]['RMSE']:.4f}")


Training Decision Tree Regressor...
✓ Decision Tree trained
  R² Score: 0.7172
  RMSE: $2.5710


### 3.6 Random Forest Regressor

In [13]:
print("\nTraining Random Forest Regressor...")

# Train with optimal parameters
rf = RandomForestRegressor(
    n_estimators=50,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train_scaled)

# Predict
y_pred_scaled = rf.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# Store
models['Random Forest'] = rf
predictions['Random Forest'] = y_pred
results.append(calculate_metrics(y_test, y_pred, 'Random Forest'))

print(f"✓ Random Forest trained")
print(f"  R² Score: {results[-1]['R² Score']:.4f}")
print(f"  RMSE: ${results[-1]['RMSE']:.4f}")


Training Random Forest Regressor...
✓ Random Forest trained
  R² Score: 0.6876
  RMSE: $2.7020
✓ Random Forest trained
  R² Score: 0.6876
  RMSE: $2.7020


## 4. Results Comparison

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('R² Score', ascending=False).reset_index(drop=True)

print("\n" + "="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

# Display the dataframe
results_df


MODEL PERFORMANCE COMPARISON
                Model      MSE     RMSE      MAE  R² Score  MAPE (%)
    Linear Regression 0.348132 0.590027 0.353121  0.985103  0.149181
     Ridge Regression 0.348756 0.590556 0.353510  0.985077  0.149347
     Lasso Regression 0.362674 0.602225 0.362410  0.984481  0.153160
Polynomial Regression 0.432431 0.657595 0.393345  0.981496  0.165808
        Decision Tree 6.609864 2.570966 1.601609  0.717162  0.667861
        Random Forest 7.300719 2.701984 1.685423  0.687600  0.702109


AttributeError: The '.style' accessor requires jinja2

## 5. Visualizations

### 5.1 Performance Metrics Comparison

In [ ]:
# Create comprehensive comparison plots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Model Performance Metrics Comparison', fontsize=16, fontweight='bold')

metrics = ['MSE', 'RMSE', 'MAE', 'R² Score', 'MAPE (%)']
colors = plt.cm.Set3(range(len(results_df)))

# Plot each metric
for idx, metric in enumerate(metrics):
    ax = axes[idx // 3, idx % 3]
    
    sorted_df = results_df.sort_values(metric, ascending=(metric != 'R² Score'))
    
    bars = ax.barh(sorted_df['Model'], sorted_df[metric], color=colors)
    ax.set_xlabel(metric, fontweight='bold')
    ax.set_title(f'{metric} by Model', fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, sorted_df[metric])):
        if metric == 'MAPE (%)':
            label = f'{val:.2f}'
        else:
            label = f'{val:.4f}'
        ax.text(val, bar.get_y() + bar.get_height()/2, label, 
               va='center', ha='left', fontsize=9, fontweight='bold')

# Remove empty subplot
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

### 5.2 R² Score Ranking

In [ ]:
# Create R² Score comparison
plt.figure(figsize=(12, 6))

sorted_df = results_df.sort_values('R² Score')
colors = ['#ff6b6b' if x < 0.5 else '#ffd93d' if x < 0.7 else '#51cf66' 
          for x in sorted_df['R² Score']]

bars = plt.barh(sorted_df['Model'], sorted_df['R² Score'], color=colors, edgecolor='black', linewidth=1.5)

plt.xlabel('R² Score', fontsize=12, fontweight='bold')
plt.title('Model Accuracy Ranking (R² Score)', fontsize=14, fontweight='bold')
plt.xlim(0, 1)
plt.grid(axis='x', alpha=0.3)

# Add value labels
for bar, val in zip(bars, sorted_df['R² Score']):
    plt.text(val + 0.02, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
            va='center', fontweight='bold', fontsize=11)

# Add reference lines
plt.axvline(x=0.7, color='green', linestyle='--', alpha=0.5, label='Good (0.7)')
plt.axvline(x=0.5, color='orange', linestyle='--', alpha=0.5, label='Fair (0.5)')
plt.legend()

plt.tight_layout()
plt.show()

# Identify best model
best_model = results_df.iloc[0]
print(f"\n🏆 BEST MODEL: {best_model['Model']}")
print(f"   R² Score: {best_model['R² Score']:.4f}")
print(f"   RMSE: ${best_model['RMSE']:.4f}")
print(f"   MAE: ${best_model['MAE']:.4f}")

### 5.3 Prediction vs Actual Comparison

In [ ]:
# Plot predictions vs actual for all models
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Predicted vs Actual Prices - All Models', fontsize=16, fontweight='bold')

# Use only last 200 points for clarity
plot_range = slice(-200, None)
y_test_plot = y_test.values[plot_range]
x_axis = range(len(y_test_plot))

for idx, (model_name, y_pred) in enumerate(predictions.items()):
    ax = axes[idx // 3, idx % 3]
    
    y_pred_plot = y_pred[plot_range]
    
    ax.plot(x_axis, y_test_plot, label='Actual', linewidth=2, alpha=0.7, color='blue')
    ax.plot(x_axis, y_pred_plot, label='Predicted', linewidth=2, alpha=0.7, color='red', linestyle='--')
    
    ax.set_title(f'{model_name}', fontweight='bold', fontsize=12)
    ax.set_xlabel('Time Steps', fontweight='bold')
    ax.set_ylabel('Price ($)', fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Add R² score to plot
    r2 = results_df[results_df['Model'] == model_name]['R² Score'].values[0]
    ax.text(0.02, 0.98, f'R² = {r2:.4f}', transform=ax.transAxes, 
           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
           fontweight='bold')

plt.tight_layout()
plt.show()

### 5.4 Error Distribution Analysis

In [ ]:
# Calculate errors for each model
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Prediction Error Distribution', fontsize=16, fontweight='bold')

for idx, (model_name, y_pred) in enumerate(predictions.items()):
    ax = axes[idx // 3, idx % 3]
    
    errors = y_test.values - y_pred
    
    ax.hist(errors, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Error')
    ax.set_title(f'{model_name}', fontweight='bold')
    ax.set_xlabel('Prediction Error ($)', fontweight='bold')
    ax.set_ylabel('Frequency', fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Add statistics
    mean_error = np.mean(errors)
    std_error = np.std(errors)
    ax.text(0.02, 0.98, f'Mean: ${mean_error:.3f}\nStd: ${std_error:.3f}', 
           transform=ax.transAxes, verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
           fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

### 5.5 Feature Importance (for Tree-based Models)

In [ ]:
# Feature importance for Decision Tree and Random Forest
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Feature Importance Analysis', fontsize=16, fontweight='bold')

tree_models = ['Decision Tree', 'Random Forest']

for idx, model_name in enumerate(tree_models):
    model = models[model_name]
    importance = model.feature_importances_
    
    # Sort by importance
    indices = np.argsort(importance)[::-1][:10]  # Top 10 features
    top_features = [features[i] for i in indices]
    top_importance = importance[indices]
    
    ax = axes[idx]
    bars = ax.barh(top_features, top_importance, color='teal', edgecolor='black')
    ax.set_xlabel('Importance', fontweight='bold')
    ax.set_title(f'{model_name} - Top 10 Features', fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, top_importance):
        ax.text(val, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
               va='center', ha='left', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Summary Statistics Table

In [ ]:
# Create a detailed summary table
summary_df = results_df.copy()

# Add ranking
summary_df['Rank'] = range(1, len(summary_df) + 1)

# Reorder columns
summary_df = summary_df[['Rank', 'Model', 'R² Score', 'RMSE', 'MAE', 'MSE', 'MAPE (%)']]

print("\n" + "="*90)
print("FINAL MODEL RANKING & PERFORMANCE SUMMARY")
print("="*90)
print(summary_df.to_string(index=False))
print("="*90)

# Display the table
summary_df

## 7. Findings & Recommendations

In [ ]:
best_model_name = results_df.iloc[0]['Model']
best_r2 = results_df.iloc[0]['R² Score']
best_rmse = results_df.iloc[0]['RMSE']
best_mae = results_df.iloc[0]['MAE']

print("="*90)
print("KEY FINDINGS & RECOMMENDATIONS")
print("="*90)
print(f"\n🏆 BEST PERFORMING MODEL: {best_model_name}")
print(f"\n   Performance Metrics:")
print(f"   - R² Score: {best_r2:.4f} (explains {best_r2*100:.2f}% of variance)")
print(f"   - RMSE: ${best_rmse:.4f} (average prediction error)")
print(f"   - MAE: ${best_mae:.4f} (average absolute error)")
print(f"\n📊 MODEL COMPARISON:")
for idx, row in results_df.iterrows():
    print(f"   {idx+1}. {row['Model']}: R² = {row['R² Score']:.4f}, RMSE = ${row['RMSE']:.4f}")

print(f"\n💡 INSIGHTS:")
print(f"   - Linear models (Linear, Ridge, Lasso) perform similarly")
print(f"   - Polynomial features may improve or worsen performance")
print(f"   - Tree-based models can capture non-linear patterns")
print(f"   - Random Forest generally reduces overfitting vs Decision Tree")

print(f"\n✅ RECOMMENDATION:")
if best_r2 > 0.7:
    recommendation = "EXCELLENT - Use this model for production"
elif best_r2 > 0.5:
    recommendation = "GOOD - Model is viable but can be improved"
else:
    recommendation = "FAIR - Consider feature engineering or more data"

print(f"   {recommendation}")
print(f"   Switch to {best_model_name} for best performance!")
print("\n" + "="*90)

## 8. Export Results

In [ ]:
# Save results to CSV for report
summary_df.to_csv('model_comparison_results.csv', index=False)
print("✓ Results exported to 'model_comparison_results.csv'")

# Save best model (if you want to switch)
import joblib

if best_model_name != 'Polynomial Regression':
    best_model_obj = models[best_model_name]
    joblib.dump(best_model_obj, f'best_model_{best_model_name.replace(" ", "_").lower()}.joblib')
    print(f"✓ Best model saved to 'best_model_{best_model_name.replace(' ', '_').lower()}.joblib'")

print("\n✅ Analysis Complete!")